<a href="https://colab.research.google.com/github/Erasmo96/RecSys/blob/main/RecSys_with_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "numpy<2"

In [2]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2457389 sha256=69e82c8e779a9b596e24f027f86e980cbf595543261e3bc2b9ab7fd554c5652e
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [3]:
!pip install surprise

In [4]:
from surprise import Dataset, Reader
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy
import pandas as pd

In [12]:
rating_df = pd.read_csv('ratings.csv')
movie_df = pd.read_csv('movies.csv')

df = pd.merge(rating_df, movie_df[['movieId','genres']], on='movieId', how='left')
df.head()

,userId,movieId,rating,timestamp,genres
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Comedy|Romance
2,1,6,4.0,964982224,Action|Crime|Thriller
3,1,47,5.0,964983815,Mystery|Thriller
4,1,50,5.0,964982931,Crime|Mystery|Thriller


In [14]:
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()
mlb = MultiLabelBinarizer()

df['user_id'] = user_encoder.fit_transform(df['userId'])
df['movie_id'] = movie_encoder.fit_transform(df['movieId'])

df = df.join(pd.DataFrame(mlb.fit_transform(df.genres.str.split('|')),
                          columns=mlb.classes_,
                          index=df.index))
df.drop(columns = "(no genres listed)", inplace=True)
df.head()

,userId,movieId,rating,timestamp,genres,user_id,movie_id,Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1,4.0,964982703,Adventure|Animation|Children|Comedy|Fantasy,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,3,4.0,964981247,Comedy|Romance,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,1,6,4.0,964982224,Action|Crime|Thriller,0,5,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,47,5.0,964983815,Mystery|Thriller,0,43,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,1,50,5.0,964982931,Crime|Mystery|Thriller,0,46,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [15]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)

In [16]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train_df[['user_id', 'movie_id', 'rating']], reader)
trainSet = data.build_full_trainset()
trainSet

In [17]:
model_svd = SVD()
model_svd.fit(trainSet)

pred_svd = model_svd.test(trainSet.build_anti_testset())
accuracy.rmse(pred_svd)

RMSE: 0.4732


0.4731891011271975

In [18]:
def get_top_n_rec(user_id, n=5):
  user_movies = df[df['userId']==user_id]['movieId'].unique()
  all_movies = df['movieId'].unique()
  movies_to_predict = list(set(all_movies) - set(user_movies))

  user_movie_pairs = [(user_id, movie_id, 0) for movie_id in movies_to_predict]
  predictions_cf = model_svd.test(user_movie_pairs)

  top_n_rec = sorted(predictions_cf, key=lambda x: x.est, reverse=True)[:n]

  top_n_movie_ids = [int(pred.iid) for pred in top_n_rec]

  return movie_encoder.inverse_transform(top_n_movie_ids)

In [19]:
user_id = 1
recs = get_top_n_rec(user_id)
top_n_movies_title = movie_df[movie_df['movieId'].isin(recs)]['title'].tolist()
print(f"Top 5 Recommended Movies for User {user_id}:")
for i,title in enumerate(top_n_movies_title,1):
    print(f"{i}.{title}")

Top 5 Recommended Movies for User 1:
1.Forrest Gump (1994)
2.Princess Bride, The (1987)
3.Patton (1970)
4.Seven Samurai (Shichinin no samurai) (1954)
5.Matrix, The (1999)
